In [ ]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [ ]:
path = r"Uke_31_dual_flowbased_values\output_fast_oneyear\1991.h5"

In [ ]:
h5py.File(path).keys()

In [ ]:
def byte_column_names_to_str(df):
    rename_mapping = dict(zip(df.columns, df.columns.str.decode("utf-8")))
    df = df.rename(columns = rename_mapping)
    return df

def processing(path, x_field, y_field):
    df2 = pd.DataFrame(np.array(h5py.File(path)[y_field]))
    df2 = df2.transpose()
    df = pd.DataFrame(np.array(h5py.File(path)[x_field]))
    df2.columns = df[0].tolist()
    df2 = byte_column_names_to_str(df2)
    return df2

def net_balance(path, supply_or_demand, f):
    detailed = f"{supply_or_demand}names"
    balance = f"{supply_or_demand}balancenames"
    values = f"{supply_or_demand}values"

    df_balance = processing(path, detailed, balance)
    df = processing(path, detailed, values)
    
    df = df.T
    df["balancename"] = df_balance.T
    df = df[[f(index) for index in df.index]]
    df = df.groupby("balancename").sum().T
    
    df = byte_column_names_to_str(df)
    return df

In [ ]:
df_demand = processing(path, "demandbalancenames", "demandvalues")
df_demand = df_demand.T.groupby(df_demand.columns).sum().T

df_supply = processing(path, "supplybalancenames", 'supplyvalues')
df_supply = df_supply.T.groupby(df_supply.columns).sum().T

df_prices = processing(path,  "areanames", "pricematrix") * 100 # euro/10mwh to euro/mwh
df_flowbased = processing(path,  "flowbasednames", "flowbasedvalues")

df_demandnames = processing(path, "demandnames", "demandvalues")
df_supplynames = processing(path, "supplynames", "supplyvalues")

In [ ]:
f = lambda x : ("Transm" in x) or ("Response" in x)
df_transm_out = net_balance(path, "demand", f) # out from balance
df_transm_in = net_balance(path, "supply", f) # in to balance

f = lambda x : ("Transm" not in x) and ("Response" not in x)
df_sup = net_balance(path, "supply", f)
df_dem = net_balance(path, "demand", f)

df_supply = df_sup + df_transm_out * 0
df_demand = df_dem + df_transm_in * 0

df_supply.columns.name = ""
df_demand.columns.name = ""

In [ ]:
fin = lambda x : "in" in x
fout = lambda x : "out" in x
fborder = lambda x : not ("in" in x or "out" in x)
name_convert = lambda x, name: "Transm_" + x.split("_")[1].replace(name, "")
in_lookup = dict([(name_convert(e, "in"), e) for e in df_flowbased.columns if fin(e)])
out_lookup = dict([(name_convert(e, "out"), e) for e in df_flowbased.columns if fout(e)])
border_lookup = dict([(name_convert(e, ""), e) for e in df_flowbased.columns if fborder(e)])

def get_lookup_value(lookup, transfer):
    # checks both direction A->B, B->A
    rev = lambda x: "Transm_" + "->".join(reversed(x.replace("Transm_", "").split("->")))
    res = lookup.get(rev(transfer), None)
    if not res:
        res = lookup.get(rev(transfer), None)
    return res

In [ ]:
t = 600

transfers = [e for e in df_demandnames.columns if "Transm" in str(e)]
areas0 = [tr.replace("Transm_", "").split("->")[0] for tr in transfers]
areas1 = [tr.replace("Transm_", "").split("->")[1] for tr in transfers]

transfer_values_demand = [df_demandnames[transfer][t] for transfer in transfers]
transfer_values_supply = [df_supplynames[transfer][t] for transfer in transfers]

border_flowbased_values = [df_flowbased.iloc[t].get( get_lookup_value(border_lookup, transfer) ) for transfer in transfers]
in_flowbased_values = [df_flowbased.iloc[t].get( get_lookup_value(in_lookup, transfer) ) for transfer in transfers]
out_flowbased_values = [df_flowbased.iloc[t].get( get_lookup_value(out_lookup, transfer) ) for transfer in transfers]

prices_area0 = df_prices.iloc[t][areas0].tolist()
demand_area0 = df_demand.iloc[t][areas0].tolist()
supply_area0 = df_supply.iloc[t][areas0].tolist()
prices_area1 = df_prices.iloc[t][areas1].tolist()
demand_area1 = df_demand.iloc[t][areas1].tolist()
supply_area1 = df_supply.iloc[t][areas1].tolist()

NP_area0 = [(df_transm_out - df_transm_in)[area][t] for area in areas0]
NP_area1 = [(df_transm_out - df_transm_in)[area][t] for area in areas1]

area0_name = [area for area in areas0]
area1_name = [area for area in areas1]
transfer_name = [transfer for transfer in transfers]

m = pd.DataFrame(
    [
        transfer_values_demand,
        transfer_values_supply,
        border_flowbased_values,
        in_flowbased_values,
        out_flowbased_values,
        prices_area0 ,
        prices_area1 ,
        NP_area0,
        NP_area1,
        demand_area0 ,
        demand_area1 ,
        supply_area0 ,
        supply_area1, 
        area0_name, 
        area1_name, 
        transfer_name
    ]
).T

m.columns = [
    "transfer_values_demand",
    "transfer_values_supply",
    "border_flowbased_values",
    "in_flowbased_values",
    "out_flowbased_values",
    "prices_area0" ,
    "prices_area1" ,
    "NP_area0",
    "NP_area1",
    "demand_area0" ,
    "demand_area1" ,
    "supply_area0" ,
    "supply_area1", 
    "area0_name", 
    "area1_name", 
    "transfer_name"
]

In [ ]:
# border_flowbased_values: constraints that keeps the flow variable for transmissions the same as defined by PTDF table
# in_flowbased_values: constraints for line using RAM, going positive direction
# out_flowbased_values: constraints for line using RAM, going negative direction
# None flowbased means NTC capacity

m

## Example

In [ ]:
m[m["area0_name"] == "NORGEMIDT"]

In [ ]:
m[m["area1_name"] == "NORGEMIDT"]

In [ ]:
m[m["area1_name"] == "NORGEMIDT"]["NP_area1"].iloc[0]
# NP value means the netto transfer need to go to NORGEMIDT

In [ ]:
m[m["area0_name"] == "NORGEMIDT"]["supply_area0"].iloc[0] - m[m["area0_name"] == "NORGEMIDT"]["demand_area0"].iloc[0]
# production - supply for NORGEMIDT

In [ ]:
sum(m[m["area0_name"] == "NORGEMIDT"]["transfer_values_demand"]) 
# Transfer going out from NORGEMIDT
# Note: this would also be taken from transfer going into NORGEMIDT from outside

In [ ]:
sum(m[m["area1_name"] == "NORGEMIDT"]["transfer_values_supply"])
# Transfer going into NORGEMIDT

In [ ]:
# Which gives the nettoposition

## Problems

In [ ]:
# Possible problems:
# Static PTDF values can restrict the flow of power to certain part of the network which could go against the optimal solution

# Possible Solutions:
# Might need to be able to go back and forth if its a dead end node? if not some percentage of power will be lost.
# Can not have static PTDF, but recreate PTDF based on demand and generation?